In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [14]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-37429?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Retail Associate, PT

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Associate, PTShrewsbury, New JerseyBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [16]:
res.content

'[\n  {\n    "role": "Retail Associate, PT",\n    "experience": "Part Time – 20 - 38 hours per week, including nights and weekends",\n    "skills": [\n      "Customer service",\n      "Product knowledge",\n      "Ability to learn and train on the latest products and technologies",\n      "Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time"\n    ],\n    "description": "Become a Part of the NIKE, Inc. Team. NIKE, Inc. does more than outfit the world’s best athletes. It is a place to explore potential, obliterate boundaries and push out the edges of what can be. The company looks for people who can grow, think, dream and create. Its culture thrives by embracing diversity and rewarding imagination. The brand seeks achievers, leaders and visionaries. At NIKE, Inc. it’s about each person bringing skills and passion to a challenging and constantly evolving game."\n  }\n]'

In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Retail Associate, PT',
  'experience': 'Part Time – 20 - 38 hours per week, including nights and weekends',
  'skills': ['Customer service',
   'Product knowledge',
   'Ability to learn and train on the latest products and technologies',
   'Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time'],
  'description': 'Become a Part of the NIKE, Inc. Team. NIKE, Inc. does more than outfit the world’s best athletes. It is a place to explore potential, obliterate boundaries and push out the edges of what can be. The company looks for people who can grow, think, dream and create. Its culture thrives by embracing diversity and rewarding imagination. The brand seeks achievers, leaders and visionaries. At NIKE, Inc. it’s about each person bringing skills and passion to a challenging and constantly evolving game.'}]

In [18]:
type(json_res)

list

In [19]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [25]:
links = collection.query(query_texts=['Get me profiles having python skill set','Also get me the ones having react native skill set'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [28]:
json_res[0]

{'role': 'Retail Associate, PT',
 'experience': 'Part Time – 20 - 38 hours per week, including nights and weekends',
 'skills': ['Customer service',
  'Product knowledge',
  'Ability to learn and train on the latest products and technologies',
  'Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time'],
 'description': 'Become a Part of the NIKE, Inc. Team. NIKE, Inc. does more than outfit the world’s best athletes. It is a place to explore potential, obliterate boundaries and push out the edges of what can be. The company looks for people who can grow, think, dream and create. Its culture thrives by embracing diversity and rewarding imagination. The brand seeks achievers, leaders and visionaries. At NIKE, Inc. it’s about each person bringing skills and passion to a challenging and constantly evolving game.'}

In [29]:
job = json_res[0]
job['skills']

['Customer service',
 'Product knowledge',
 'Ability to learn and train on the latest products and technologies',
 'Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time']

In [30]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [32]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Sandesh, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Your Retail Operations with AtliQ's Automation Solutions

Dear Hiring Manager,

I came across the job posting for a Retail Associate, PT at NIKE, Inc. and was impressed by the company's commitment to innovation and customer satisfaction. As a Business Development Executive at AtliQ, I believe our company can help take your retail operations to the next level with our automation solutions.

At AtliQ, we specialize in developing tailored software solutions that streamline business processes, optimize efficiency, and reduce costs. Our team of experts has extensive experience in creating customized tools that cater to the unique needs of retail businesses like NIKE, Inc.

Our portfolio showcases our capabilities in developing innovative solutions using various technologies. For instance, our work in machine learning and Python ([https://example.com/ml-python-portfolio](https://example.com/ml-python-portfolio)) demonstrates our ability to create intelligent systems that can